#### As importing the csv file in MySQL did not worked properly, I had to clean and analyse the data set only with Python. 
#### Without any error, only a part of the rows have been imported. 
#### Here are some queries, I would have used to explore the data. 

In [2]:
import pandas as pd
import getpass
import sqlalchemy as sa

In [3]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'  

Your password:········


In [4]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [5]:
db_connection = sa.create_engine(connection_string)

In [6]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()

In [7]:
query= "SELECT * FROM travel.travel;"
q = pd.read_sql_query(query, db_connection)
q.head()

,user_ID,search_ts,user_city_source2,user_country_source2,website_language,days_to_departure,trip_duration,searched_destination,distance
0,AAE/wjE8vdKAM/5wIgm+lzMggnQ=,2019-08-06 00:26:59,Dammam,SA,ar,15,5,Sharjah,1751.40
1,AAJxNQjR5b+EpApPE9CfR0i5ew4=,2019-08-14 11:08:23,Dubai,AE,en,45,37,Kathmandu,2990.55
2,AAJxNQjR5b+EpApPE9CfR0i5ew4=,2019-08-14 11:08:23,Dubai,AE,en,42,38,Kathmandu,2990.55
3,AAPohIYHePQvSMeyL1q3hqilhBI=,2019-08-29 21:40:20,Dammam,SA,ar,7,3,Cairo,2415.46
4,AAPohIYHePQvSMeyL1q3hqilhBI=,2019-08-29 21:40:20,Dammam,SA,ar,7,14,Cairo,2415.46


#### Destinations

In [8]:
query= """
SELECT COUNT(DISTINCT(searched_destination)) as number_of_destinations
FROM travel.travel;
"""
q = pd.read_sql_query(query, db_connection)
q

,number_of_destinations
0,310


In [9]:
query= """
SELECT user_country_source2,
        COUNT(DISTINCT(searched_destination)) as number_of_searched_destinations_per_user_country
FROM travel.travel
GROUP BY user_country_source2
ORDER BY number_of_searched_destinations_per_user_country DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_country_source2,number_of_searched_destinations_per_user_country
0,AE,108
1,GB,95
2,SA,86
3,FR,74
4,DE,63
...,...,...
65,SK,1
66,TH,1
67,MT,1
68,AL,1


Users from AE searched for 108 different destinations and thus the most different destinations.

#### User City

In [10]:
query= """
SELECT COUNT(DISTINCT(user_city_source2)) as number_of_user_cities
FROM travel.travel;
"""
q = pd.read_sql_query(query, db_connection)
q

,number_of_user_cities
0,590


In [14]:
query= """
SELECT user_city_source2,  
        COUNT(user_city_source2) AS number_of_searches_for_destinations
FROM travel.travel 
GROUP BY user_city_source2 
ORDER BY COUNT(user_city_source2) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_city_source2,number_of_searches_for_destinations
0,Dubai,763
1,Jeddah,396
2,Riyadh,299
3,Abu Dhabi,139
4,Sharjah,91
...,...,...
585,Abbiategrasso,1
586,Ibbenbueren,1
587,Wesseling,1
588,'s-Hertogenbosch,1


The most searches came from Dubai 

In [15]:
query= """
SELECT user_city_source2,
       ROUND((COUNT(user_city_source2)*100)/
           (
           SELECT count(user_city_source2) FROM travel.travel),2
           ) 
       AS percentage_of_all_searched_cities
FROM travel.travel
GROUP BY user_city_source2
ORDER BY COUNT(user_city_source2) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q.head(10)

,user_city_source2,percentage_of_all_searched_cities
0,Dubai,17.77
1,Jeddah,9.22
2,Riyadh,6.96
3,Abu Dhabi,3.24
4,Sharjah,2.12
5,Kuwait City,1.68
6,Cairo,1.54
7,Al Ain,1.47
8,Dammam,1.30
9,Dhahran,1.30


18% of all searches came from users from Dubai

In [98]:
query= """
SELECT user_city_source2,
   COUNT(searched_destination) as overall_number_of_searches_for_destinations
FROM travel.travel 
WHERE user_city_source2 = 'Jeddah'
GROUP BY user_city_source2 
ORDER BY COUNT(searched_destination) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_city_source2,overall_number_of_searches_for_destinations
0,Jeddah,396


From Jeddah 396 searches were conducted.

In [71]:
query= """
SELECT user_city_source2,
   COUNT(DISTINCT(searched_destination)) as number_of_unique_destinations
FROM travel.travel 
WHERE user_city_source2 = 'Jeddah'
GROUP BY user_city_source2 
ORDER BY COUNT(searched_destination) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_city_source2,number_of_unique_destinations
0,Jeddah,57


From user city Jeddah, 57 different destinations were searched

In [100]:
query= """
SELECT user_city_source2,
   COUNT(DISTINCT(searched_destination)) as number_of_unique_destinations
FROM travel.travel 
WHERE user_city_source2 = 'Dubai'
GROUP BY user_city_source2 
ORDER BY COUNT(searched_destination) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_city_source2,number_of_unique_destinations
0,Dubai,90


Users from Duabi searched for 90 different destinations

#### Trip Duration

In [81]:
query= """
SELECT CEILING(AVG(trip_duration)) AS average_trip_duration_of_all_trips
FROM travel.travel;
"""
q = pd.read_sql_query(query, db_connection)
q

,average_trip_duration_of_all_trips
0,14


In [95]:
query= """
SELECT searched_destination, CEILING(AVG(trip_duration)) AS average_trip_duration_of_this_destination
FROM travel.travel
GROUP BY searched_destination
HAVING average_trip_duration_of_this_destination >= 14
ORDER BY average_trip_duration_of_this_destination DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,searched_destination,average_trip_duration_of_this_destination
0,BogotÃƒÂ¡,140
1,Stockholm,84
2,Port Harcourt,77
3,Chittagong,73
4,Lagos,49
...,...,...
113,Nuremberg,14
114,Lima,14
115,Tel Aviv,14
116,Pantelleria,14


In [91]:
query= """
SELECT searched_destination, CEILING(AVG(trip_duration)) AS average_trip_duration
FROM travel.travel
WHERE searched_destination = 'Abu Dhabi';
"""
q = pd.read_sql_query(query, db_connection)
q

,searched_destination,average_trip_duration
0,Abu Dhabi,12


#### Languages

In [29]:
query= """
SELECT REPLACE('website_language', ' ', '')
"""
q = pd.read_sql_query(query, db_connection)
q

,"REPLACE('website_language', ' ', '')"
0,website_language


In [94]:
query= """
SELECT website_language, count(website_language) AS count_of_searches_in_this_languages
FROM travel.travel
GROUP BY website_language
ORDER BY count_of_searches_in_this_languages DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,website_language,count_of_searches_in_this_languages
0,en,1905
1,ar,1372
2,fr,279
3,[%.,221
4,it,183
5,de,96
6,es,83
7,pt,62
8,nl,39
9,ru,18


The most searches were conducted in an english website language, followed by arabic

#### User ID

In [146]:
query= """
SELECT user_ID, COUNT(user_ID) as count_of_searches
FROM travel.travel
GROUP BY user_ID
ORDER BY count_of_searches DESC;
"""
q = pd.read_sql_query(query, db_connection)
q

,user_ID,count_of_searches
0,Ay01MmLpxei4NhLTopMKq3euKCA=,127
1,BnqgW//Kj/NkeuNyc121Rg9flJc=,96
2,AlHaCly5714nsygxNWm9tSXyI/Y=,73
3,BmujTHLfDQkTrEabZCd2n8mLS1o=,61
4,AAXQks+ZkbFWKMfaYayfp6RS58o=,51
...,...,...
1226,Bot7EMFEui57j/CyVk1Lam+lI7w=,1
1227,Bo8vvyzOABrV1IDQfkC8JAn1RXg=,1
1228,Bo/chPKfJZxRMsnGDo43w30LvMc=,1
1229,BpAnDauHnz3LgSnPYO1wCvhOKOo=,1


#### Days to Departure

In [155]:
query= """
SELECT days_to_departure, COUNT(days_to_departure) AS count_of_frequency
FROM travel.travel
GROUP BY days_to_departure
ORDER BY COUNT(days_to_departure) DESC;
"""
q = pd.read_sql_query(query, db_connection)
q.head(10)


,days_to_departure,count_of_frequency
0,2,246
1,4,244
2,5,231
3,3,211
4,1,208
5,7,177
6,6,154
7,8,142
8,10,142
9,11,128


For most of the searches, there are 2 or 4 days to departure 

In [156]:
query= """
SELECT CEILING(AVG(days_to_departure)) AS overall_average_days_to_departure
FROM travel.travel;
"""
q = pd.read_sql_query(query, db_connection)
q

,overall_average_days_to_departure
0,40


In [169]:
query= """
SELECT days_to_departure, 
        ROUND(days_to_departure*100/
            (SELECT SUM(days_to_departure) FROM travel.travel),3) 
        AS percentage
FROM travel.travel
GROUP BY days_to_departure
ORDER BY days_to_departure;
"""
q = pd.read_sql_query(query, db_connection)
q.head(10)


,days_to_departure,percentage
0,1,0.001
1,2,0.001
2,3,0.002
3,4,0.002
4,5,0.003
5,6,0.004
6,7,0.004
7,8,0.005
8,9,0.005
9,10,0.006


#### Challenges while importing csv in MySQL

Action: using 'LOAD DATA LOCAL INFILE' 

Error Code: 3948. Loading local data is disabled; this must be enabled on both the client and server 

Handling:
Try setting it from MySQL command line
SHOW GLOBAL VARIABLES LIKE 'local_infile';    and then set it to true:  SET GLOBAL local_infile = true;

In [ ]:
Error Code: 2068. LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.

Error Code: 1290. The MySQL server is running with the –secure-file-priv option so it cannot execute this statement”.

Handling:
as the file path to run LOAD DATA as this is the configured directory and he needs to move the data file here.

In the end 5000 rows have been imported into the table. With no error about the other rows. 